In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time
    
# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [11]:
flujos_loc = pd.read_csv('tablas/dd_localidades.csv')
print(flujos_loc.shape[0])
flujos_loc.head(3)

377610


,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.0,583715
1,10202521,1020,2521,1304729,12200,90.0,612802
2,10202522,1020,2522,1304729,2659,17.0,617076


In [12]:
locs = cargar_datos_geo()[2][['CODLOC', 'geometry']]
locs.head(3)

,CODLOC,geometry
0,2220,POINT (549273.219 6633915.063)
1,2521,POINT (442454.671 6652273.541)
2,2522,POINT (457921.001 6633361.660)


In [13]:
locs_cod = list(locs.CODLOC)
len(locs_cod) ** 2 - len(locs_cod)

377610

In [14]:
# pega geometrías
dd = flujos_loc.merge(locs, left_on='cod_ori', right_on='CODLOC')
dd = dd.merge(locs, left_on='cod_des', right_on='CODLOC')
dd.drop(['CODLOC_x', 'CODLOC_y'], axis=1, inplace=True)
dd.rename({'geometry_x':'geom_ori', 'geometry_y':'geom_des'}, axis=1, inplace=True)

dd.head()

# exporta CSV con geometrías puntuales para mapa de flujos
#dd.to_csv('capas/loc_lines.csv', index=False)

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m,geom_ori,geom_des
0,10202220,1020,2220,1304729,40657,635.0,583715,POINT (573082.248 6145008.434),POINT (549273.219 6633915.063)
1,25212220,2521,2220,12200,40657,NaN,137927,POINT (442454.671 6652273.541),POINT (549273.219 6633915.063)
2,25222220,2522,2220,2659,40657,NaN,111408,POINT (457921.001 6633361.660),POINT (549273.219 6633915.063)
3,26212220,2621,2220,2531,40657,NaN,106867,POINT (468458.270 6601361.867),POINT (549273.219 6633915.063)
4,27212220,2721,2220,380,40657,NaN,58392,POINT (502880.928 6648162.053),POINT (549273.219 6633915.063)


In [15]:
print(flujos_loc.shape)
flujos_loc.head()

(377610, 7)


,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.0,583715
1,10202521,1020,2521,1304729,12200,90.0,612802
2,10202522,1020,2522,1304729,2659,17.0,617076
3,10202621,1020,2621,1304729,2531,9.0,595785
4,10202721,1020,2721,1304729,380,NaN,628213


In [16]:
# reemplaza ceros
def nozeros(df):
    df_copy = df
    df_copy.loc[df_copy.personas_mig.isna(), 'personas_mig'] = 0.001
    df_copy.loc[df_copy.poblacion_des==0, 'poblacion_des'] = 1
    df_copy.loc[df_copy.poblacion_ori==0, 'poblacion_ori'] = 1
    return df

In [17]:
flujos_loc = nozeros(flujos_loc)
flujos_loc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.000,583715
1,10202521,1020,2521,1304729,12200,90.000,612802
2,10202522,1020,2522,1304729,2659,17.000,617076
3,10202621,1020,2621,1304729,2531,9.000,595785
4,10202721,1020,2721,1304729,380,0.001,628213


In [18]:
# convierte códigos a string para ser correctamente interpretados por el modelo
flujos_loc['cod_ori'] = flujos_loc['cod_ori'].astype(str)
flujos_loc['cod_des'] = flujos_loc['cod_des'].astype(str)

flujos_loc_oc = flujos_loc

flujos_loc_oc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.000,583715
1,10202521,1020,2521,1304729,12200,90.000,612802
2,10202522,1020,2522,1304729,2659,17.000,617076
3,10202621,1020,2621,1304729,2531,9.000,595785
4,10202721,1020,2721,1304729,380,0.001,628213


In [19]:
formula = "personas_mig ~ cod_ori + np.log(poblacion_des) + np.log(distancia_m) -1"

t0= time.time()

# entrena y ajusta el modelo
prodSim = smf.glm(formula=formula, data=flujos_loc_oc, family=sm.families.Poisson()).fit(method='lbfgs', max_start_irls=0)

t1 = time.time() - t0
print(t1)

C:\Users\user\anaconda3\envs\tesis\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
106.569491147995


In [78]:
#prodSim.summary()
prodSim.params

cod_ori[1020]            6.905786
cod_ori[10320]           3.896025
cod_ori[10321]           2.599674
cod_ori[10521]           0.229187
cod_ori[10522]           0.507811
                           ...   
cod_ori[9957]           -0.568802
cod_ori[9958]           -0.533280
cod_ori[9991]           -0.381858
np.log(poblacion_des)    0.921008
np.log(distancia_m)     -0.865461
Length: 617, dtype: float64

In [79]:
# recupera los parámetros del modelo
mu_i = prodSim.params.to_frame()

# elimina caractérres no numéricos para poder pegar
mu_i.rename(index = dict(zip(mu_i.index[0:-2].values,  mu_i.index[0:-2].str.replace('cod_ori[','', regex=False).str.replace(']','', regex=False).values)),
            inplace=True)

# renombre columna
mu_i.rename(columns = {0:'mu_i'}, inplace=True)

# merge
flujos_loc_oc = flujos_loc_oc.merge(mu_i, left_on='cod_ori', right_index=True)

flujos_loc_oc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m,mu_i
0,10202220,1020,2220,1304729,40657,635.000,583715,6.905786
1,10202521,1020,2521,1304729,12200,90.000,612802,6.905786
2,10202522,1020,2522,1304729,2659,17.000,617076,6.905786
3,10202621,1020,2621,1304729,2531,9.000,595785,6.905786
4,10202721,1020,2721,1304729,380,0.001,628213,6.905786


In [80]:
# funciones para recuperar parámetros e imprimir
def get_gml_params(model, variables):
    "Accede a los parámetros alfa y beta dentro de los resutaldos del modelo"
    params = [model.params[i] for i in variables]
    params_str = [str(round(i, 4)) for i in params]
    return params, params_str

def print_params(variables, params_list):
    "Imprime los parámetros"
    return print("""alpha ({}) = {}\nbeta ({}) = {}
    """.format(variables[0], params_list[0], variables[1], params_list[1]))

In [81]:
# imprime parámetros
variables= ['np.log(poblacion_des)', 'np.log(distancia_m)']

params, params_str = get_gml_params(prodSim, variables)

print_params(variables, params_str)

alpha (np.log(poblacion_des)) = 0.921
beta (np.log(distancia_m)) = -0.8655
    


In [82]:
variables

['np.log(poblacion_des)', 'np.log(distancia_m)']

In [83]:
# genera estimación redondeada
def prod_sim_est(df, variables, alpha, beta):
    "Estimación del modelo imputando los parámetros alfa y beta previamente calculados"
    prodsimest = np.exp(df['mu_i'] + alpha * np.log(df[variables[0]]) + beta * np.log(df[variables[1]]))
    return round(prodsimest)

In [84]:
# recupera alpha y beta, previamente guardados en la lista de parámetros
alpha, beta = [i for i in params]

variables = ['poblacion_des', 'distancia_m']

# estima y guarda en columna "podsimtest"
flujos_loc_oc['prodsimest'] = prod_sim_est(flujos_loc_oc, variables, alpha, beta)

In [85]:
flujos_loc_oc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m,mu_i,prodsimest
0,10202220,1020,2220,1304729,40657,635.000,583715,6.905786,179.0
1,10202521,1020,2521,1304729,12200,90.000,612802,6.905786,57.0
2,10202522,1020,2522,1304729,2659,17.000,617076,6.905786,14.0
3,10202621,1020,2621,1304729,2531,9.000,595785,6.905786,14.0
4,10202721,1020,2721,1304729,380,0.001,628213,6.905786,2.0


In [86]:
# matriz de flujos estimada por el modelo
flujos_loc_oc['cod_ori'] = flujos_loc_oc.cod_ori.astype(int)
flujos_loc_oc['cod_des'] = flujos_loc_oc.cod_des.astype(int)

matrix_prodsim = pd.pivot_table(flujos_loc_oc,
                                values='prodsimest',
                                index ='cod_ori',
                                columns='cod_des',
                                fill_value=0,
                                aggfunc=sum,
                                margins=True,
                                margins_name='Total')

matrix_prodsim.Total = matrix_prodsim.Total.astype(int)

#matrix_prodsim

In [87]:
# crea la sumatoria de migrantes en origen (Oi)
O_i = flujos_loc_oc.groupby('cod_ori')['personas_mig'].sum().to_frame().rename(columns = {'personas_mig':'O_i'})

# crea la sumatoria de migrantes en destino (Di)
D_j = flujos_loc_oc.groupby('cod_des')['personas_mig'].sum().to_frame().rename(columns = {'personas_mig':'D_j'})

# pega ambas variables con el df
flujos_loc_oc = flujos_loc_oc.merge(O_i, left_on='cod_ori', right_index=True)
flujos_loc_oc = flujos_loc_oc.merge(D_j, left_on='cod_des', right_index=True)

flujos_loc_oc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m,mu_i,prodsimest,O_i,D_j
0,10202220,1020,2220,1304729,40657,635.000,583715,6.905786,179.0,55484.172,1043.544
1229,25212220,2521,2220,12200,40657,0.001,137927,4.726421,71.0,1172.518,1043.544
1843,25222220,2522,2220,2659,40657,0.001,111408,0.638936,1.0,144.595,1043.544
2457,26212220,2621,2220,2531,40657,0.001,106867,0.911650,2.0,191.580,1043.544
3071,27212220,2721,2220,380,40657,0.001,58392,-0.147396,1.0,12.609,1043.544


In [30]:
# bondad de ajuste

# define una función para calcular el R cuadrado
def calcR2(obs, est):
    return np.power(np.corrcoef(obs,est),2.0)[0][1]

# define una función para calcula el error medio cuadrático
def calcRMSE(obs,est):
    return np.sqrt((np.power((obs - est),2.0)).mean())

In [89]:
printmd('**Bondad de ajuste del modelo restringido en origen**')

printmd("$R²$ = " + round(calcR2(flujos_loc['personas_mig'],flujos_loc['prodsimest']), 4).astype(str))

printmd("RMSE = " + round(calcRMSE(flujos_loc['personas_mig'],flujos_loc['prodsimest']), 4).astype(str))


**Bondad de ajuste del modelo restringido en origen**

$R²$ = 0.5221

RMSE = 12.6171

# Modelo de doble restrición

In [20]:
flujos_loc_dc = flujos_loc
flujos_loc_dc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m
0,10202220,1020,2220,1304729,40657,635.000,583715
1,10202521,1020,2521,1304729,12200,90.000,612802
2,10202522,1020,2522,1304729,2659,17.000,617076
3,10202621,1020,2621,1304729,2531,9.000,595785
4,10202721,1020,2721,1304729,380,0.001,628213


In [23]:
formula = "personas_mig ~ cod_ori + np.log(poblacion_des) + np.log(distancia_m) -1"

t0= time.time()

# entrena y ajusta el modelo
doubSim = smf.glm(formula=formula, data=flujos_loc_dc, family=sm.families.Poisson()).fit(method='lbfgs', max_start_irls=0)

t1 = time.time() - t0
print(t1)

C:\Users\user\anaconda3\envs\tesis\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
125.52654266357422


In [24]:
# recupera los valores estimados
flujos_loc_dc['doubsim_ajustado'] = np.round(doubSim.predict())

flujos_loc_dc['cod_ori'] = flujos_loc_dc.cod_ori.astype(int)

In [28]:
flujos_loc_dc.doubsim_ajustado.max()

7444.0

In [31]:
printmd('**Bondad de ajuste del modelo de doble restricción**')

printmd("$R²$ = " + round(calcR2(flujos_loc_dc['personas_mig'],  flujos_loc_dc['doubsim_ajustado']), 4).astype(str))

printmd("RMSE = " + round(calcRMSE(flujos_loc_dc['personas_mig'], flujos_loc_dc['doubsim_ajustado']), 4).astype(str))

**Bondad de ajuste del modelo de doble restricción**

$R²$ = 0.5221

RMSE = 12.6171

In [32]:
# basado en: https://journals.sagepub.com/doi/abs/10.1177/030913257900300218
# Here is the entropy maximising approach for a known beta.
# Plug in the required values in this function to solve.

def balance_doubly_constrained(df, orig_field, dest_field,
                               Oi_field, Dj_field, cij_field, beta, 
                               cost_function,
                               Ai_name = "Ai_new", Bj_name = "Bj_new", converge=0.001):
    # Define some variables
    Oi = df[[orig_field, Oi_field]]
    Dj = df[[dest_field,Dj_field]]
    
    if cost_function.lower() in ['power','pow']:
        beta_cij = np.exp(beta * np.log(df[cij_field]))
    elif cost_function.lower() in ['exponential','exp']:
        beta_cij = np.exp(beta * df[cij_field])
    else:
        return "Cost function not specified properly, use 'exp' or 'pow'"
    
    # Create some helper variables
    cnvg = 1
    iteration = 0
    
    # Now iteratively rebalance the Ai and Bj terms until convergence
    while cnvg > converge:
        if iteration == 0:
            # This first condition sets starting values for Ai and Bj
            # NB sets starting value of Ai assuming Bj is a vector of 1s.
            # We've already established beta_cij with the appropriate cost function, so...
            Oi = Oi.assign(Ai = Dj[Dj_field] * beta_cij)
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Now set up Bjs using starting values of Ai
            Dj = Dj.assign(Bj = Oi['Ai_old'] * Oi[Oi_field] * beta_cij)
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Increment loop
            iteration += 1
        else:
            # This bit is the iterated bit of the loop which refines the values of Ai and Bj
            # First Ai
            Oi['Ai'] = Dj['Bj_old'] * Dj[Dj_field] * beta_cij
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Drop temporary Ai
            Oi.drop('Ai', axis=1, inplace=True)
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True)
            # Calculate the difference between old and new Ais
            Oi['diff'] = np.absolute((Oi['Ai_old'] - Oi['Ai'])/Oi['Ai_old'])
            # Set new Ais to Ai_old
            Oi['Ai_old'] = Oi['Ai']
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Then Bj
            Dj['Bj'] = Oi['Ai_old'] * Oi[Oi_field] * beta_cij
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Drop temporary Bj
            Dj.drop('Bj', axis=1, inplace=True)
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True)
            # Calculate the difference between old and new Bjs
            Dj['diff'] = np.absolute((Dj['Bj_old'] - Dj['Bj'])/Dj['Bj_old'])
            # Set new Bjs to Bj_old
            Dj['Bj_old'] = Dj['Bj']
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Assign higher sum difference from Ai or Bj to cnvg
            cnvg = np.maximum(Oi['diff'].sum(),Dj['diff'].sum())
            
            # Print and increment loop
            print("Iteration:", iteration)
            iteration += 1

    # When the while loop finishes add the computed Ai_old and Bj_old to the dataframe and return
    df[Ai_name] = Oi['Ai_old']
    df[Bj_name] = Dj['Bj_old']
    return df

In [33]:
# create some Oi and Dj columns in the dataframe and store row and column totals in them:
# First get the origin sums and rename the column created
O_i = flujos_loc_dc.groupby('cod_ori')['personas_mig'].sum().to_frame()
O_i.rename(columns = {'personas_mig':'O_i'}, inplace=True)

# Now get the destination sums
D_j = flujos_loc_dc.groupby('cod_des')['personas_mig'].sum().to_frame()
D_j.rename(columns = {'personas_mig':'D_j'}, inplace=True)

# Merge in O_i
flujos_loc_dc = flujos_loc_dc.merge(O_i,left_on='cod_ori', right_index=True)

# Merge in D_j
flujos_loc_dc = flujos_loc_dc.merge(D_j,left_on='cod_des', right_index=True)

flujos_loc_dc.head()

,cod,cod_ori,cod_des,poblacion_ori,poblacion_des,personas_mig,distancia_m,doubsim_ajustado,O_i,D_j
0,10202220,1020,2220,1304729,40657,635.000,583715,179.0,55484.172,1043.544
1229,25212220,2521,2220,12200,40657,0.001,137927,71.0,1172.518,1043.544
1843,25222220,2522,2220,2659,40657,0.001,111408,1.0,144.595,1043.544
2457,26212220,2621,2220,2531,40657,0.001,106867,2.0,191.580,1043.544
3071,27212220,2721,2220,380,40657,0.001,58392,1.0,12.609,1043.544


In [34]:
# recupera el beta del logaritmo de la distancia del modelo anterior
beta = doubSim.params[-1]
beta

-0.8654606427983192

#zero inflated
from patsy import dmatrices


df = flujos_loc

expr = "personas_mig ~ cod_ori + log_dist -1"

mask = np.random.rand(len(df)) < 0.8
df_train = df[mask]
df_test = df[~mask]
print('Training data set length='+str(len(df_train)))
print('Testing data set length='+str(len(df_test)))


y_train, X_train = dmatrices(expr, df_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, df_test, return_type='dataframe')

zip_training_results = sm.ZeroInflatedPoisson(endog=y_train, exog=X_train, exog_infl=X_train, inflation='logit').fit()

zip_training_results.summary()

zip_predictions = zip_training_results.predict(X_test,exog_infl=X_test)

predicted_counts=np.round(zip_predictions)

actual_counts = y_test['personas_mig']

print('ZIP RMSE='+str(np.sqrt(np.sum(np.power(np.subtract(predicted_counts,actual_counts),2)))))



fig = plt.figure(figsize=(12, 8), dpi=150)

fig.suptitle('Predicted versus actual counts using the ZIP model')

predicted, = plt.plot(X_test.index, predicted_counts, 'g', label='Predicted')

actual, = plt.plot(X_test.index, actual_counts, 'r', label='Actual')

plt.legend(handles=[predicted, actual])

plt.show()